**Cargue de librerias**

In [75]:
import pandas as pd
import os
import sys
import asyncio
from playwright.async_api import async_playwright
from pathlib import Path


**Cargue Data**

In [77]:
#generar archivo resultados simulacion.py
ruta_carpeta = ("../02_Querys")  # cambiar ruta para que se generica

# Verificar si la carpeta existe
if not os.path.exists(ruta_carpeta):
    print(f" Error: La carpeta '{ruta_carpeta}' no existe. Verifica la ruta.")
else:
    # Nombre del archivo
    archivo = "Resultados_simulacion.py"

    # Ruta completa del archivo
    ruta_archivo = os.path.join(ruta_carpeta, archivo)


    # Guardar el DataFrame en el archivo .py
    with open(ruta_archivo, "w") as f:
        f.write("import pandas as pd\n\n")  # Importación en el archivo
        #f.write(f"df = {df.to_dict()}\n")   # Guarda el DataFrame como un diccionario

    print(f"Archivo guardado en: {ruta_archivo}")

Archivo guardado en: ../02_Querys\Resultados_simulacion.py


**Funcion aplicar tasa**

In [79]:

# Definir la función para calcular la tasa
def calcular_tasa(P, A, n, tol=1e-6, max_iter=1000):
    
    #Calcula la tasa de interés mensual y anual de un préstamo.
 
    r = 0.05  # Suposición inicial (5%)
    for _ in range(max_iter):
        f_r = P - (A * (1 - (1 + r) ** -n) / r)
        f_r_deriv = (A * ((1 + r) ** -n * n / (1 + r) - 1) / r**2)
        
        if abs(f_r) < tol:
            break
        
        r -= f_r / f_r_deriv
    
    # Calcular tasas anuales
    tasa_efectiva_anual = (1 + r) ** 12 - 1
    
    return tasa_efectiva_anual, r

#  Ruta del archivo Excel de entrada
ruta_entrada = ("../03_Resultados/Resultados_tasa.xlsx")  # Cambia la ruta

# Cargar los datos desde el archivo Excel
df_resultados_global = pd.read_excel(ruta_entrada)

#  Verificar que las columnas necesarias existen
columnas_requeridas = {"ValorCredito", "valor_cuota", "Plazo"}
if not columnas_requeridas.issubset(df_resultados_global.columns):
    raise ValueError(f" Faltan columnas necesarias en el archivo Excel: {columnas_requeridas - set(df_resultados_global.columns)}")

#  Aplicar la función de tasa de interés a cada fila
df_resultados_global[["TEA", "Tasa_Mensual"]] = df_resultados_global.apply(
    lambda row: calcular_tasa(row["ValorCredito"], row["valor_cuota"], row["Plazo"]),
    axis=1, result_type="expand"
)

df_resultados_global['TEA']=df_resultados_global['TEA'].fillna(0)
df_resultados_global['Tasa_Mensual']=df_resultados_global['Tasa_Mensual'].fillna(0)

# Definir la ruta del archivo de salida
ruta_salida =  ("../03_Resultados/tasas_calculadas.xlsx")

# Guardar el DataFrame con los resultados en un nuevo archivo Excel
df_resultados_global.to_excel(ruta_salida, index=False, sheet_name="Tasas")

print(f"Archivo Excel con tasas guardado en: {ruta_salida}")


Archivo Excel con tasas guardado en: ../03_Resultados/tasas_calculadas.xlsx


C:\Users\jojha\AppData\Local\Temp\ipykernel_19864\1253902049.py:14: RuntimeWarning: divide by zero encountered in scalar divide
  r -= f_r / f_r_deriv
C:\Users\jojha\AppData\Local\Temp\ipykernel_19864\1253902049.py:14: RuntimeWarning: invalid value encountered in scalar subtract
  r -= f_r / f_r_deriv


In [80]:
df_resultados_global

,ValorCredito,Plazo,valor_cuota,TEA,Tasa_Mensual
0,2000000,12,196038,0.359590,0.025929
1,2000000,24,112941,0.359136,0.025900
2,2000000,36,86091,0.359133,0.025900
3,2000000,48,0,0.000000,0.000000
4,3000000,24,169411,0.359131,0.025900
5,3000000,36,129136,0.359129,0.025900
6,4500000,6,0,0.000000,0.000000
7,4500000,24,254116,0.359129,0.025900
8,10000000,24,564703,0.359131,0.025900
9,10000000,36,430453,0.359129,0.025900
